In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial


import pprint
import numpy as np

def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id):
    return Evaluation((__noworkflow__.trial_id, evaluation_id))

def print_act_forward_act(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward_act(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        
        
def module(num):
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean((x - y))
    
def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))


def recursive_act(act_list, indent):
    current_trial = __noworkflow__.trial_id
    
    if act_list:
        act = act_list.pop(0)
        if act.type in ('func', 'assign'):           
            if act.type == 'func':
                sub_act = Activation((current_trial, act.dependent_id))
                print(' ' * indent + "actv id = "+ str(sub_act.id) + '; function = ' + str(sub_act.name) + '; code_block = ' + str(sub_act.code_block_id))
                
            if act.type == 'assign':
                sub_eval = Evaluation((current_trial, act.dependent_id))
                code_comp = sub_eval.code_component
                if (code_comp.type == 'call') or ((code_comp.mode == 'w') and (code_comp.type != 'param')): 
                    op_name = code_comp.name
                    print(' ' * indent + "eval id = "+ str(code_comp.id) + "; variable = " + code_comp.name + ' <- ' + str(sub_eval.repr))
        recursive_act(act_list, indent)
    else:
        print("- - - - -")

In [2]:
def module(num):
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)


In [3]:
x = 10
y = -100

In [4]:
rme_err = mae(x, y)

In [5]:
z = 50

In [6]:
rmse_err = rmse(x, rme_err)

In [7]:
k = module(rmse_err)
k_activation = __noworkflow__.last_activation

In [8]:
k_activation

Activation(id=67, name=<ipython-input-7-afe7de53980c>, start_checkpoint=6.836792495999999, code_block_id=694)

In [9]:
a = __noworkflow__.first_activation

In [10]:

evaluation = get_evaluation(1)
querier = DependencyQuerier()
nodes_to_visit, visited, found = querier.navigate_dependencies([evaluation])

for i in visited:
    a, _ = i.evaluation.code_component, i.steps
    print(a)


code_component(1a5e7404-4a81-4839-a253-3a5c3016eace, 1, '.', 'notebook', 'w', 1, 0, 1, 0, nil).


[]

In [35]:
list(a.dependent_dependencies)

[dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 68, 20, 21, 'assignment', 1, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 69, 47, 66, 'assignment', 1, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 70, 70, 79, 'use', 1, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 70, 70, 74, 'condition', 0, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 70, 67, 68, 'func', 0, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 70, 67, 69, 'argument', 0, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 80, 67, 70, 'assign', 1, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 83, 67, 81, 'value', 0, nil, nil, nil).,
 dependency(81d23e54-d004-4226-853e-d433e338af53, 67, 85, 67, 83, 'assign', 1, nil, nil, nil).]

In [72]:
a = get_activation(70)

In [73]:
for i in list(a.dependent_dependencies):
    print(type(i), i.type, i.reference)

<class 'noworkflow.now.persistence.models.dependency.Dependency'> argument True
<class 'noworkflow.now.persistence.models.dependency.Dependency'> assignment True
<class 'noworkflow.now.persistence.models.dependency.Dependency'> use False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> use False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> assignment True
<class 'noworkflow.now.persistence.models.dependency.Dependency'> condition False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> assignment True
<class 'noworkflow.now.persistence.models.dependency.Dependency'> condition False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> func False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> argument False
<class 'noworkflow.now.persistence.models.dependency.Dependency'> dependency False


In [74]:
i.prolog_description

dependency(TrialId, DependentActivationId, DependentId, DependencyActivationId, DependencyId, Type, Reference, CollectionActivationId, CollectionId, Key).

In [62]:
i.reference

True

In [16]:
trial_id = __noworkflow__.trial_id

base_act = get_activation(67, trial_id)
act_list = list(base_act.dependent_dependencies)

indent = 1

for i in act_list:
        print(i.dependent_id)
        if i.type not in ('assignment', 'value', 'argument', 'dependency', 'assign'):
            sub_act = get_activation(i.dependent_id, trial_id)
            #sub_act = get_evaluation(i.dependent_id)
            print('passou')
            if sub_act.has_activations: 
                print(' ' * indent + "actv id = "+ str(sub_act.id) + '; function = ' + str(sub_act.name))
                act_list.extend(sub_act.dependent_dependencies)
                #print(len(act_list))
            else:
                print(' ' * indent + "actv id = "+ str(sub_act.id) + '; function = ' + str(sub_act.name) + '; code_block = ' + str(sub_act.type))
 

68
69
70
passou
 actv id = 70; function = module
70
passou
 actv id = 70; function = module
70
passou
 actv id = 70; function = module
70
80
83
85
71
72
74


AttributeError: 'NoneType' object has no attribute 'trial_id'

In [17]:
print_act_evals(1)

OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 2),
             ('checkpoint', 6.3108665859999995),
             ('code_component_id', 2),
             ('activation_id', 1),
             ('repr', 'None'),
             ('member_container_activation_id', 1),
             ('member_container_id', 2)])
OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 20),
             ('checkpoint', 6.417165555),
             ('code_component_id', 617),
             ('activation_id', 1),
             ('repr', 'None'),
             ('member_container_activation_id', 1),
             ('member_container_id', 20)])
OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 22),
             ('checkpoint', 6.492864377),
             ('code_component_id', 653),
             ('activation_id', 1),
             ('repr', 'None'),
             ('member_container_activation_id', 1),
             ('member_container_

In [22]:
print_act_forward_act(1)

OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 68),
             ('dependent_activation_id', 1),
             ('dependent_id', 86),
             ('dependency_activation_id', 86),
             ('dependency_id', 87),
             ('type', 'assign'),
             ('reference', True),
             ('collection_activation_id', None),
             ('collection_id', None),
             ('key', None)])
OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 113),
             ('dependent_activation_id', 1),
             ('dependent_id', 141),
             ('dependency_activation_id', 141),
             ('dependency_id', 142),
             ('type', 'assign'),
             ('reference', True),
             ('collection_activation_id', None),
             ('collection_id', None),
             ('key', None)])
OrderedDict([('trial_id', '1a5e7404-4a81-4839-a253-3a5c3016eace'),
             ('id', 117),
             ('dependent_ac

Observations:

1. The function is recursive in only one direction. Trying to search backwards will break it
2. It only displays the history of what was executed inside the cell. It doesn't attends the need of printing the dependencies of
    a given variable. 

The usecase here is:
When tagging one variable, I want to be able to capture all the dependencies of this variable accross the trial.
1. I have to iterate over all the executed cells in the notebook
2. Capture all references to this variabble
3. Capture all references to the references of this variable
4. DIsplay it at the end.